# 1. Import

In [1]:
from rag_utils import minsearch_index, elasticsearch_index, search, elastic_search, build_prompt, llm

from dotenv import load_dotenv
from openai import OpenAI

import os

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
file_path = './documents.json'

# 2. Minsearch

In [3]:
index = minsearch_index(file_path)
index_boost = {'question': 3.0, 'section': 0.5}
index_filter = {'course': 'data-engineering-zoomcamp'}

In [4]:
q = 'the course has already started, can I still enroll?'

def minsearch_rag(query):
    search_results = search(index, query, index_boost, index_filter, top_k=5)
    prompt = build_prompt(query, search_results)
    answer = llm(client, prompt, model_name='gpt-4o')
    return {'search_results': search_results, 'prompt': prompt, 'answer': answer}

minsearch_response = minsearch_rag(q)
minsearch_response['answer']

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you don't register before the course begins. However, be sure to adhere to the deadlines for submitting the final projects to avoid any issues."

# 3. Elasticsearch

In [5]:
es_client = elasticsearch_index(file_path, 'http://localhost:9200')

  0%|          | 0/948 [00:00<?, ?it/s]

In [6]:
q = 'the course has already started, can I still enroll?'

def elastic_rag(query): 
    search_results = elastic_search(query, es_client, index_name='course-questions')
    prompt = build_prompt(query, search_results)
    answer = llm(client, prompt, model_name='gpt-4o')
    return {'search_results': search_results, 'prompt': prompt, 'answer': answer}

elasticsearch_response = elastic_rag(q)
elasticsearch_response['answer']

"Yes, you can still enroll in the course after it has started. You can submit homework even if you haven't registered. However, be mindful of the deadlines for the final projects and avoid leaving everything until the last minute."